In [1]:
import torch

from tqdm import tqdm

from transformers import OPTForCausalLM

In [2]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!nvidia-smi

Mon Feb 27 14:34:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   27C    P0    35W / 250W |      2MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

# Sparsify Model

In [4]:
from utils.prune_utils import mask_lowest
from utils.save_utils import mask_from_pruned
from torch.nn.utils import prune
from utils.fsdp_finetune import fsdp_finetune

for model_name in ['opt-125m', 'opt-350m', 'opt-1.3b']:
    for SPARSENESS in [0.2,0.3,0.5,0.7,0.9,1]:
        # Load model with pre-trained head
        model = OPTForCausalLM.from_pretrained(f'facebook/{model_name}',
                                               output_attentions=True,
                                               output_hidden_states=True).to(device=device) # type: ignore
        model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])
        mask_lowest(model, amount=1-SPARSENESS, prune_remove=False)
        config = {"model": model, "lr": 2e-5, "num_epochs": 1,
                  "seed": 1, "batch_size": 16,
                  'model_name': model_name,
                  'sparsity': 1-SPARSENESS,"train_steps": 10000,
                  'max_step': 10000, 'save_model': False}
        fsdp_finetune(config)
        pruned_model_name = f'{model_name}-cerebras-{SPARSENESS}'
        torch.save(model, f'pruned_models/{pruned_model_name}-finetuned.pt')

DistributedType.NO


  0%|          | 0/1 [00:00<?, ?it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/cuda/memory.py:282: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(

  0%|          | 0/10000 [00:00<?, ?it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

  3%|▎         | 289/10000 [00:19<08:42, 18.57it/s]


  6%|▌         | 599/10000 [00:35<07:59, 19.61it/s]


  9%|▉         | 914/10000 [00:51<07:52, 19.24it/s]


 12%|█▏        | 1218/10000 [01:07<07:37, 19.18it/s]


...

 96%|█████████▌| 9560/10000 [08:19<00:22, 19.59it/s]


 99%|█████████▊| 9860/10000 [08:35<00:07, 19.38it/s]


100%|██████████| 1/1 [08:43<00:00, 523.10s/it]


Memory before entering the train : 1129
Memory consumed at the end of the train (end-begin): 1761
Peak Memory consumed during the train (max-begin): 3133
Total Peak Memory consumed during the train (max): 4262


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_4159/2585223663.py:20 in <cell line: 6>                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4159/2585223663.py'                  │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'get_prop_zeros' is not defined